In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf
import pathlib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv3D, Flatten, Dropout, MaxPooling3D

AUTOTUNE = tf.data.experimental.AUTOTUNE

print("Tensorflow Version: ", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Tensorflow Version:  2.1.0
Num GPUs Available:  1


In [2]:
path = pathlib.Path('export/data/dataset.npz')
with np.load(path, mmap_mode="r") as data:
    train_examples = data['dataset'][:310]
    train_labels = data['labels'][:310]
    val_examples = data['dataset'][311:347]
    val_labels = data['labels'][311:347]
    test_examples = data['dataset'][347:386]
    test_labels = data['labels'][347:386]

train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_examples, val_labels))

In [3]:
BATCH_SIZE = 4
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

In [4]:
model = Sequential([
    Conv3D(16, (1, 4, 3), activation='relu', input_shape=(15, 240, 320, 3)),
    MaxPooling3D(),
    Conv3D(32, (1, 4, 3), activation='relu'),
    MaxPooling3D(),
    Conv3D(64, (1, 4, 3), activation='relu'),
    MaxPooling3D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 15, 237, 318, 16)  592       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 118, 159, 16)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 7, 115, 157, 32)   6176      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 57, 78, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 3, 54, 76, 64)     24640     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 27, 38, 64)     0         
_________________________________________________________________
flatten (Flatten)            (None, 65664)             0

In [5]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(train_dataset, epochs=100, callbacks=[callback], validation_data=val_dataset)
model.evaluate(test_dataset)

Train for 78 steps, validate for 9 steps
Epoch 1/100
78/78 [==============================] - 28s 361ms/step - loss: 0.9677 - accuracy: 0.5387 - val_loss: 0.6889 - val_accuracy: 0.5556
Epoch 2/100
78/78 [==============================] - 19s 245ms/step - loss: 0.6914 - accuracy: 0.5871 - val_loss: 0.6818 - val_accuracy: 0.5556
Epoch 3/100
78/78 [==============================] - 18s 231ms/step - loss: 0.6407 - accuracy: 0.6129 - val_loss: 0.6211 - val_accuracy: 0.7778
Epoch 4/100
78/78 [==============================] - 18s 228ms/step - loss: 0.6279 - accuracy: 0.6871 - val_loss: 0.6463 - val_accuracy: 0.7222
Epoch 5/100
78/78 [==============================] - 18s 228ms/step - loss: 0.5718 - accuracy: 0.7194 - val_loss: 0.6315 - val_accuracy: 0.6667
Epoch 6/100
78/78 [==============================] - 18s 231ms/step - loss: 0.5212 - accuracy: 0.7516 - val_loss: 0.5760 - val_accuracy: 0.7222
Epoch 7/100
78/78 [==============================] - 18s 228ms/step - loss: 0.4342 - accuracy: 

[0.6522874236106873, 0.71794873]

In [6]:
export_path = pathlib.Path("export/model.h5")
print("Exporting trained model to {}".format(export_path.absolute()))

model.save(export_path)

Exporting trained model to d:\Dev\Projects\SpeechActionClassifier\export\model.h5
